In [1]:
import mido
from mido import MidiFile, Message, MidiTrack
import torch
import os
import numpy as np
import time

In [2]:
def CountTracks(directory):          #Count files and tracks in folder
    trackCount = 0
    fileCount = 0
    for file in os.listdir(directory):
        if file.endswith(".midi"):
            fileCount += 1
            midiDir = MidiFile(directory+"/"+file)
            for track in midiDir.tracks:
                trackCount += 1
    print(fileCount+" files")
    print(trackCount+" tracks")

    
def PrintMessages(mid):                # print midi messages
    for i, track in enumerate(mid.tracks):
        print('Track {}: {}'.format(i, track.name))
        for msg in track:
            print(msg)

            
def PrintSomeMessages(mid):             #print first 200 midi messages
    track = mid.tracks[1]
    for i,msg in enumerate(track):
        if i < 200:
            print(msg)

In [3]:
def Midi2Numpy(mid):                                #converts to numpy array removing non-note messages
    track = mid.tracks[1]                           #0th track only contains meta-messages, all notes on 1st track
    notes = np.empty([0,4])
    time = 0
    for msg in track:
        if msg.type == "note_on":                   # only count "note" messages - other inputs i.e. foot pedals are ignored
            notes = np.append(notes,np.array([[msg.note, msg.velocity, msg.time + time, 0]]),axis=0)         # (note, velocity, time, sustain)
            time = 0
        else:
            time += msg.time                        #adjust time when removing other messages
    return notes


def NumpyGetSustain(notes):
    for i, msg in enumerate(notes):
        if msg[1] > 0:                            # if velocity is not 0
            j = 1
            sustain = 0
            while msg[0] != notes[i+j][0]:        # while note values are different
                sustain += notes[i+j][2]
                j += 1                            #search for next message with same note i.e. message telling that note was released
            notes[i,3] = sustain + notes[i+j][2]
    time = 0
    for i, msg in enumerate(notes):
        if msg[1] > 0:
            notes[i,2] += time
            time = 0
        else:
            time += msg[2]                        #adjust time
    notes = notes[notes[:,1] > 0]                 #filter for notes with positive velocities (note presses)
    return notes

In [12]:
def cleanupMessages(mid):
    track = mid.tracks[1]
    track2 = []
    for msg in track:
        if msg.type == "note_on":
            track2.append(msg)
    mid.tracks[1] = track2

In [16]:
mid = MidiFile('testSong.midi')
#cleanupMessages(mid)
#mid.save("processed.midi")
PrintSomeMessages(mid)

<meta message track_name name='7.6.13 Group 1' time=0>
program_change channel=0 program=0 time=0
control_change channel=0 control=64 value=95 time=0
control_change channel=0 control=67 value=0 time=0
control_change channel=0 control=64 value=95 time=46
control_change channel=0 control=64 value=94 time=11
control_change channel=0 control=64 value=91 time=15
control_change channel=0 control=64 value=87 time=16
control_change channel=0 control=64 value=83 time=15
control_change channel=0 control=64 value=79 time=16
control_change channel=0 control=64 value=75 time=15
control_change channel=0 control=64 value=71 time=15
control_change channel=0 control=64 value=68 time=16
control_change channel=0 control=64 value=65 time=15
control_change channel=0 control=64 value=64 time=15
control_change channel=0 control=64 value=63 time=15
control_change channel=0 control=64 value=62 time=15
control_change channel=0 control=64 value=62 time=16
control_change channel=0 control=64 value=61 time=15
contr

In [ ]:
t0 = time.time()

mid = MidiFile('testSong.midi')
notes = Midi2Numpy(mid)
cleanNotes = NumpyGetSustain(notes)
np.savetxt("test.csv", cleanNotes, delimiter=",")

t1 = time.time()
print(t1-t0)

In [ ]:
def CSV2Numpy(file):
    notes = np.getfromtxt(file,delimiter=",")
    for i,n in enumerate(notes):
        


In [ ]:
a = np.array([3,5,7])
b = np.copy(a)
b[1] = 984
print(a)
print(b)